# Fine-Tunning

Cuando tenemos pocos datos, y además no hay una red especifica que comparta mucho dominio con nuestro caso de uso, podemos hacer fine-tuning a una red ya existente.

El proceso de finetuning consiste en hacer freeze de ciertas capas, y entrenar otras, además de entrenar la cabeza.

Dependiendo de que características queramos quedarnos (mas o menos generales) y del grado de dominio que compartamos entre la red source y nuestro target, haremos freeze de más o menos capas.

En general este es un proceso iterativo, prueba y error.

Las capas iniciales capturan patrones genéricos (bordes, texturas), útiles para casi cualquier tarea.

Las capas profundas capturan rasgos específicos del dataset original, por lo que son las que suele convenir reajustar cuando cambias de dominio.

El fine-tuning siempre incluye:

-  Sustituir la cabeza (head) por una nueva adaptada al nº de clases objetivo.

- Decidir qué capas del backbone se reentrenan y cuáles se congelan.

- Usar una tasa de aprendizaje pequeña (a menudo 10× menor que en entrenamiento desde cero) para no destruir pesos    preentrenados.

Empecemos.

Como hicimos antes, cargamos los pesos de la red VGG16 entranada con IMGNET

In [3]:
from torchvision import models, datasets, transforms
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary

#Obtenemos los pesos de la red VGG16 entrenada en el dataset IMAGENET

weights = models.VGG16_Weights.IMAGENET1K_V1

#Como vemos aqui observamos los valores a los que nuestro dataset debe ser normalizado
# y preprocesado para que la red pueda usarlo.

preprocess = weights.transforms()
print(preprocess)
IMAGENET_MEAN = preprocess.mean
IMAGENET_STD  = preprocess.std

print(IMAGENET_MEAN)
print(IMAGENET_STD)


ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)
[0.485, 0.456, 0.406]
[0.229, 0.224, 0.225]


In [4]:


# Replicamos el pipeline de preprocesado de la red VGG16
base_tf = transforms.Compose([
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR), # Redimensiona la imagen
    transforms.CenterCrop(224), # Recorta la imagen
    transforms.ToTensor(), # Convierte la imagen a un tensor
    transforms.Normalize(weights.transforms().mean, weights.transforms().std), # Normaliza la imagen
])

root = "./data/02"
train_ds = datasets.ImageFolder(f"{root}/train", transform=base_tf)
valid_ds = datasets.ImageFolder(f"{root}/valid", transform=base_tf)
test_ds  = datasets.ImageFolder(f"{root}/test",  transform=base_tf)

loader_train = DataLoader(train_ds, batch_size=64, shuffle=True)                       
loader_val   = DataLoader(valid_ds, batch_size=64, shuffle=False)
loader_test  = DataLoader(test_ds,  batch_size=64,  shuffle=False)


In [5]:
# Definimos GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Obtenemos el numero de clases de nuestro dataset
num_classes = len(train_ds.classes)
print(num_classes)

# Definimos la base de la red VGG16
base = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).to(device)


#Vamos a imprimir los parametros de la red VGG16
# Calculamos parámetros
trainable_params = sum(p.numel() for p in base.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in base.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")



10
Trainable: 138,357,544 | Frozen: 0 | Total: 138,357,544


In [7]:

# Congelamos los parametros de la red VGG16
# Aqui le decimos que no calcule los gradientes para los parametros de la red VGG16
# Esto es importante, ya que si no lo hacemos, el modelo se entrenará desde cero
# y no aprovechará los pesos ya entrenados de la red VGG16-Imagenet
for p in base.features.parameters():
    p.requires_grad = False


# Vamos a volver a calcular los parametros de la red VGG16, para ver que se han congelado
# Calculamos parámetros
trainable_params = sum(p.numel() for p in base.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in base.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")


#Quitamos el head de la red VGG16
backbone = nn.Sequential(*(list(base.children())[:-1])) 

# Vemos que los parametros entrenables se han reducido a 0
trainable_params = sum(p.numel() for p in backbone.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in backbone.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")

print(summary(backbone, (3, 224, 224)))

Trainable: 123,642,856 | Frozen: 14,714,688 | Total: 138,357,544
Trainable: 0 | Frozen: 14,714,688 | Total: 14,714,688
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56] 

Vamos a imprimir los indices y los tipos de las capas que tenemos para saber cuales tenemos que descongelar.

In [9]:
for i, layer in enumerate(base.features):
    # comprobamos si alguno de los parámetros de la capa tiene gradiente activado
    requires_grad = any(p.requires_grad for p in layer.parameters())
    print(f"{i:02d}: {layer.__class__.__name__:<20} | requires_grad={requires_grad}")


00: Conv2d               | requires_grad=False
01: ReLU                 | requires_grad=False
02: Conv2d               | requires_grad=False
03: ReLU                 | requires_grad=False
04: MaxPool2d            | requires_grad=False
05: Conv2d               | requires_grad=False
06: ReLU                 | requires_grad=False
07: Conv2d               | requires_grad=False
08: ReLU                 | requires_grad=False
09: MaxPool2d            | requires_grad=False
10: Conv2d               | requires_grad=False
11: ReLU                 | requires_grad=False
12: Conv2d               | requires_grad=False
13: ReLU                 | requires_grad=False
14: Conv2d               | requires_grad=False
15: ReLU                 | requires_grad=False
16: MaxPool2d            | requires_grad=False
17: Conv2d               | requires_grad=False
18: ReLU                 | requires_grad=False
19: Conv2d               | requires_grad=False
20: ReLU                 | requires_grad=False
21: Conv2d   

In [10]:
# Descongelamos las últimas 3 capas convolucionales  (solo esas 3 tienen pesos: block5_conv1-3)
for idx in [24, 26, 28]:
    for p in backbone[0][idx].parameters():
        p.requires_grad = True


for i, layer in enumerate(base.features):
    # comprobamos si alguno de los parámetros de la capa tiene gradiente activado
    requires_grad = any(p.requires_grad for p in layer.parameters())
    print(f"{i:02d}: {layer.__class__.__name__:<20} | requires_grad={requires_grad}")


00: Conv2d               | requires_grad=False
01: ReLU                 | requires_grad=False
02: Conv2d               | requires_grad=False
03: ReLU                 | requires_grad=False
04: MaxPool2d            | requires_grad=False
05: Conv2d               | requires_grad=False
06: ReLU                 | requires_grad=False
07: Conv2d               | requires_grad=False
08: ReLU                 | requires_grad=False
09: MaxPool2d            | requires_grad=False
10: Conv2d               | requires_grad=False
11: ReLU                 | requires_grad=False
12: Conv2d               | requires_grad=False
13: ReLU                 | requires_grad=False
14: Conv2d               | requires_grad=False
15: ReLU                 | requires_grad=False
16: MaxPool2d            | requires_grad=False
17: Conv2d               | requires_grad=False
18: ReLU                 | requires_grad=False
19: Conv2d               | requires_grad=False
20: ReLU                 | requires_grad=False
21: Conv2d   

In [ ]:
# Imprimimos los parametros de la red VGG16 una vez descongeladas las 3 capas convolucionales
trainable_params = sum(p.numel() for p in backbone.parameters() if p.requires_grad)
total_params     = sum(p.numel() for p in backbone.parameters())
frozen_params    = total_params - trainable_params

print(f"Trainable: {trainable_params:,} | Frozen: {frozen_params:,} | Total: {total_params:,}")



Trainable: 7,079,424 | Frozen: 7,635,264 | Total: 14,714,688
